## Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
!pip3 freeze | findstr pandas

!pip3 freeze | findstr python

geopandas @ file:///tmp/build/80754af9/geopandas_1627401391374/work
pandas @ file:///C:/ci/pandas_1644984307868/work
pandas-profiling @ https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
ipython @ file:///C:/ci/ipython_1643818352265/work
ipython-genutils @ file:///tmp/build/80754af9/ipython_genutils_1606773439826/work
libarchive-c @ file:///tmp/build/80754af9/python-libarchive-c_1617780486945/work
msgpack @ file:///C:/ci/msgpack-python_1612287350784/work
python-dateutil @ file:///tmp/build/80754af9/python-dateutil_1626374649649/work
python-lsp-black @ file:///tmp/build/80754af9/python-lsp-black_1634232156041/work
python-lsp-jsonrpc==1.0.0
python-lsp-server==1.2.4
python-slugify @ file:///tmp/build/80754af9/python-slugify_1620405669636/work


In [10]:
pip list

Package                            Version
---------------------------------- --------------------
alabaster                          0.7.12
anaconda-client                    1.9.0
anaconda-navigator                 2.1.2
anaconda-project                   0.10.2
anyio                              3.5.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        21.3.0
argon2-cffi-bindings               21.2.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            5.0
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.4.0
autopep8                           1.6.0
Babel                              2.9.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1

In [15]:
!pip3 freeze > requirements.txt

### Ocultando mensagens de avisos no Pandas:

In [ ]:
import warnings

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Importando os arquivos .csv e atribuindo a variáveis:

In [ ]:
d_2019 = pd.read_csv('../Dados_csv/Ano-2019.csv', sep=';')
d_2020 = pd.read_csv('../Dados_csv/Ano-2020.csv', sep=';')
d_2021 = pd.read_csv('../Dados_csv/Ano-2021.csv', sep=';')

print('O dataset de 2019 tem', d_2019.shape[0], 'linhas e', d_2019.shape[1], 'colunas.\n'
      'O dataset de 2020 tem', d_2020.shape[0], 'linhas e', d_2020.shape[1], 'colunas.\n'
      'O dataset de 2021 tem', d_2021.shape[0], 'linhas e', d_2021.shape[1], 'colunas.')

## Unificando as bases 2019, 2020 e 2021 em um único DataFrame:

In [ ]:
d_unificado = pd.concat([d_2019,d_2020,d_2021])

print('O dataframe unificado possui', d_unificado.shape[0], 'linhas e', d_unificado.shape[1], 'colunas')

## Verificando os nomes das colunas:

In [ ]:
d_unificado.columns

   ><b>OBS:</b>A formatação do nome das colunas está no padrão "CamelCase"

## Checando o espaço físico do dataset unificado:

In [ ]:
tamanho = sum(d_unificado.memory_usage())

print('O espaço físico ocupado em MB pelo dataset unificado é de', tamanho/1000000, 'MB')

## Checando os tipos de formatos em todas as colunas:

In [ ]:
d_unificado.dtypes

><b>OBSERVAÇÕES:</b>
>>1. Verificado que a coluna 'datEmissao' está no formato <b>'Object'</b>, a mesma deverá ser convertida para o formato de data.
>>2. Verificado que as colunas que contém os valores das despesas já estão em formato <b>'float64'</b>, dispensando assim sua conversão, salvo se a formatação do separador de centavos estiver fora do padrão americano (ponto).

## Verificando o percentual de valores nulos ('NaN') no dataset unificado:

>Nessa etapa o .isnull() foi utilizado, acompanhado do .mean() para encontrar o percentual de d_copia faltantes em cada coluna deste dataset.

In [ ]:
percentual_nulos = pd.DataFrame(round(d_unificado.isnull().mean()*100,2)) # Número de dados nulos em percentual

percentual_nulos.rename(columns={0:'Percentual de Dados Nulos (%)'}) 

><b>OBSERVAÇÕES:</b>
>>Verificado altos índices de valores nulos (faltantes) nas seguintes colunas:
    >>><b>vlrRestituicao (99,93%)</b>    
    >>><b>txtTrecho (76,88%)</b>     
    >>><b>txtPassageiro (76,69%)</b>    
    >>><b>txtDescricaoEspecificacao (75,49%)</b>    
    >>><b>urlDocumento (37,06%)</b>

>>No entanto, nas colunas que se referem aos valores financeiros o índice de valores nulos é aceitável.

# Tratamento e Limpeza dos Dados

### Configurando o Pandas para exibição de todas as 31 colunas do dataset:

In [ ]:
pd.set_option('display.max_columns',31) # Setando o Pandas para exibir todas as 31 colunas

### Convertendo a coluna 'datEmissao' do tipo 'Object' para o tipo 'Datetime' em todos as fontes de dados:

In [ ]:
d_2019['datEmissao'] = pd.to_datetime(d_2019['datEmissao'], format = '%Y/%m/%d')

d_2020['datEmissao'] = pd.to_datetime(d_2020['datEmissao'], format = '%Y/%m/%d')

d_2021['datEmissao'] = pd.to_datetime(d_2021['datEmissao'], format = '%Y/%m/%d')

d_unificado['datEmissao'] = pd.to_datetime(d_unificado['datEmissao'], format = '%Y/%m/%d')

### Criando uma cópia do dataset como Boa Prática de Programação:

In [ ]:
d_copia = d_unificado

### Verificando a existência de despesas sem atribuição do partido político:

In [ ]:
sem_partido = d_copia.query('sgPartido.isnull()')

### Verificado após uma amostragem que os lançamentos sem partidos políticos, se referem a lançamentos de suas lideranças partidárias.

In [ ]:
sem_partido['txNomeParlamentar'].unique()

In [ ]:
sem_partido.sample(5)

### Obtendo os nomes únicos das Lideranças Partidárias:

In [ ]:
lista_liderancas = sem_partido['txNomeParlamentar'].value_counts()

lista_liderancas = lista_liderancas.reset_index()

lista_liderancas = pd.DataFrame(lista_liderancas)

partidos_completo = lista_liderancas['index'].str.split()

partidos = []

for i in partidos_completo:
    partidos.append(i[-1])
    
lista_liderancas['Partido'] = partidos

del(lista_liderancas['txNomeParlamentar'])

lista_liderancas.rename(columns={'index': 'Nomes_partidos'}, inplace=True)

lista_liderancas

### Atribuindo as Lideranças Partidárias aos seus respectivos partidos políticos:

In [ ]:
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSL", 'PSL', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PT", 'PT', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PTB", 'PTB', d_copia.sgPartido) 
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSDB", 'PSDB', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PODEMOS", 'PODEMOS', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO SOLIDARIEDADE", 'SOLIDARIEDADE', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSD", 'PSD', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO NOVO", 'NOVO', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO DEMOCRATAS", 'DEMOCRATAS', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO CIDADANIA", 'CIDADANIA', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO AVANTE", 'AVANTE', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PDT", 'PDT', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PSOL", 'PSOL', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PROGRESSISTAS", 'PROGRESSISTAS', d_copia.sgPartido)
d_copia['sgPartido'] = np.where(d_copia['txNomeParlamentar'] == "LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL", 'REPUBLICANOS', d_copia.sgPartido)

### Checando os dados das Lideranças que ficaram sem Partidos:

In [ ]:
d_copia[d_copia['sgPartido'].isnull()]['txNomeParlamentar'].unique()

### Filtrando apenas os valores Positivos na coluna 'vlrLiquido' 

In [ ]:
valor_positivo = d_copia.query('vlrLiquido > 0')

In [ ]:
d_copia.query('vlrLiquido < 0')['txtDescricao'].unique()

><b>OBSERVAÇÃO 1:</b> 
>>À fim de averiguarmos os valores negativos diferentes de Passagens Aéreas (cujas regras diferem dos demais lançamentos - tema explicado no dicionário das variáveis), realizamos a checagem dos lançamentos negativos e constatamos que, se tratam de inconsistências de lançamentos como o apontado abaixo do parlamentar Eduardo Bolsonaro:

In [ ]:
d_copia[(d_copia['txtDescricao']=='LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES')&(d_copia['vlrLiquido']< 0)]

><b>OBSERVAÇÃO 2:</b> 
>>Ao realizarmos a filtragem pelo Estado do Mato Grosso (MT) não identificamos nenhum lançamento negativo, à não ser os já explicados que são os valores de passagens aéreas que seguem outras regras de lançamentos (compensação).Dessa forma, reiteramos que retiramos os valores negativos do dataset filtrado.

### Filtrando apenas os lançamentos do Estado do Mato Grosso (MT) e gerando o dataset "dados_mt" que será a base de análise pela equipe de projeto.

In [ ]:
dados_mt = valor_positivo.query('sgUF == "MT"')

dados_mt.sample(10)

In [ ]:
dados_mt.head()

In [ ]:
dados_mt.query('txtFornecedor == "3T HOLDING DE PARTICIPAÇÕES LTDA-EPP"')['vlrLiquido'].sum()

In [ ]:
dados_mt.query('txtFornecedor == "3T HOLDING DE PARTICIPACOES LTDA"')['vlrLiquido'].sum()

In [ ]:
dados_mt.query('txtFornecedor.str.contains("3T")')['vlrLiquido'].sum()

In [ ]:
82680.37 - 4692.34

In [ ]:
dados_mt.shape

### Exportando o dataset para o formato .csv:

In [ ]:
#dados_mt.to_csv('teste_dados_mt.csv', index=False)

# Avaliação Estatística dos Dados Filtrados no dataframe "dados_mt"

>Começamos a avaliação estatística dos dados filtrados no dataframe 'dados_mt' com a aplicação do método Describe:

In [ ]:
dados_mt.describe().T

><b>OBSERVAÇÃO:</b> 
>>Através do método do Describe podemos observar que o dataset possui uma boa consistência de dados, quanto ao preenchimento. <br>Apenas as colunas 'vlrRestituição' e 'cpf' possuem valores faltantes.

>O box plot dá uma ideia da posição, dispersão, assimetria, caudas e dados discrepantes (outliers). <br>A posição central
é dada pela Mediana e a dispersão por IIQ (Intervalo entre Quartis).
<br>As posições relativas de Q1, Mediana e Q3 dão uma noção da simetria da distribuição.
<br>Os comprimentos das caudas são dados pelas linhas que vão do retângulo aos valores remotos e pelos valores atípicos.

<p><img src="https://miro.medium.com/max/1400/1*2c21SkzJMf3frPXPAR_gZA.png" width="800" height="400"></p>

In [ ]:
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
ax = sns.boxplot(x = 'vlrLiquido', data = dados_mt, orient = 'h')
ax.figure.set_size_inches(12,6)
ax.set_title('Variação das Despesas Parlamentares', fontsize=18)
ax.set_xlabel('Reais', fontsize=14)
ax
print()

# Análise Exploratória para Resolução das Questões Predefinidas:

## 1 - Como se comportam os gastos com cotas parlamentares ao longo do tempo? Existe alguma tendência de aumento ou redução desse custo?

><b>Obs:</b> Utilizado como referência a coluna 'vlrLiquido' conforme discrimina o Dicionário de Dados: 
     <p>O seu conteúdo representa o valor líquido do documento fiscal ou do documento que deu causa à despesa
     e será calculado pela diferença entre o Valor do Documento e o Valor da Glosa. <p>É este valor que será 
     debitado da cota do deputado. Caso o débito seja do Tipo Telefonia e o valor seja igual a zero, significa
     que a despesa foi franqueada.

## 1.1 - Obtendo as despesas no período de 2019 a 2021

Aqui a opção foi usar o método dt.to_period('M') para agrupar os períodos por mês para facilitar a visualização

In [ ]:
despesas_por_mes = dados_mt.groupby(dados_mt['datEmissao'].dt.to_period('M'))['vlrLiquido'].sum().reset_index()

despesas_por_mes

Para criação do gráfico com as datas, precisamos converter a coluna 'datEmissao' do tipo datetime para o tipo string:

In [ ]:
despesas_por_mes['datEmissao'] = despesas_por_mes['datEmissao'].astype(str)

## 1.2 - Representação gráfica das despesas ao longo do período:

In [ ]:
eixo_data = despesas_por_mes['datEmissao']

eixo_valores = despesas_por_mes['vlrLiquido'] 

fig, ax = plt.subplots(figsize=(18, 6))

# Adiconando títulos
plt.title("Evolução dos gastos de Cota para o Exercício da Atividade Parlamentar no período de 2019 a 2021 ", loc='left', fontsize=14, fontweight=0)
plt.xlabel("Período", fontsize=15)
plt.ylabel("Valor em reais", fontsize=15)
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)

# rotacionar das datas
fig.autofmt_xdate()

plt.plot(eixo_data, eixo_valores)
print()

## 2 - Qual a média de gastos por parlamentar?

In [ ]:
dados_mt.txNomeParlamentar.value_counts()

In [ ]:
a = dados_mt[['txNomeParlamentar', 'sgPartido','vlrLiquido']].groupby(['txNomeParlamentar', 'sgPartido']).mean().sort_values('vlrLiquido',ascending=False).round(2)

a.rename(columns={'vlrLiquido':'Média Despesas'}, inplace=True)

a = a.reset_index()

a

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
plt.title('Média de gastos mensais por parlamentar', fontsize=20)

sns.set(font_scale=2)
sns.barplot(x="Média Despesas", y="txNomeParlamentar",data=a.sort_values(by='Média Despesas', ascending=False)[:15]);
plt.ylabel('Parlamentar', fontsize=12)
plt.xlabel('Valor em Reais R$', fontsize=12)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)

plt.tight_layout()

## 3 - Quais foram os parlamentares que mais e que menos consumiram recursos?

In [ ]:
desp_maxima_parlamentar = dados_mt[['txNomeParlamentar','sgPartido', 'vlrLiquido']].groupby(['txNomeParlamentar', 'sgPartido']).sum().sort_values('vlrLiquido',ascending=False).round(2).reset_index()

desp_maxima_parlamentar.rename(columns={'vlrLiquido':'Despesas Máximas R$'}, inplace=True)

desp_maxima_parlamentar = desp_maxima_parlamentar.head()

desp_maxima_parlamentar.style.format({'Despesas Máximas R$':'R$ {:.2F}'}).background_gradient(cmap='Reds')

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
plt.title('Parlamentares que consumiram mais recursos no Período de 2019 a 2021', fontsize=20)

grafico = sns.barplot(y = 'txNomeParlamentar', x = 'Despesas Máximas R$', data = desp_maxima_parlamentar[:10])
grafico.bar_label(grafico.containers[0], fontsize=12)
plt.ylabel('Parlamentar', fontsize=15)
plt.xlabel('Valor em Milhões de Reais R$', fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
print()

In [ ]:
desp_minima_parlamentar = dados_mt[['txNomeParlamentar','sgPartido', 'vlrLiquido']].groupby(['txNomeParlamentar', 'sgPartido']).sum().sort_values('vlrLiquido',ascending=True).round(2).reset_index()

desp_minima_parlamentar.rename(columns={'vlrLiquido':'Despesa Mínima R$'}, inplace=True)

desp_minima_parlamentar = desp_minima_parlamentar.head()

desp_minima_parlamentar.style.format({'Despesa Mínima R$':'R$ {:.2F}'}).background_gradient(cmap='Reds')

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
plt.title('Parlamentares que consumiram menos recursos no Período de 2019 a 2021', fontsize=20)

grafico = sns.barplot(y = 'txNomeParlamentar', x = 'Despesa Mínima R$', data = desp_minima_parlamentar[-10:])
grafico.bar_label(grafico.containers[0], fontsize=12)
plt.ylabel('Parlamentar')
plt.xlabel('Valores em Reais (R$)')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
print()

## 4 - Quais são as categorias de despesas mais onerosas dentre os recursos destinados às cotas parlamentares?

In [ ]:
categorias_onerosas = dados_mt[['txtDescricao', 'vlrLiquido']].groupby('txtDescricao').sum().sort_values('vlrLiquido', ascending=False)

categorias_onerosas.rename(columns={'vlrLiquido':'Valor Gasto na Categoria'}, inplace=True)

categorias_onerosas = categorias_onerosas.reset_index() # Reset Index caso deseje gerar gráfico (ele reposiciona as colunas)

categorias_onerosas.style.format({'Valor Gasto na Categoria':'R$ {:.2F}'}).background_gradient(cmap='Reds')

## 4.1 - Plotando um gráfico com as categorias mais onerosas:

In [ ]:
# Determinando a categoria e valores:
x = categorias_onerosas.txtDescricao
y = categorias_onerosas['Valor Gasto na Categoria']/1000000

# Ajustando o tamanho da Área do Gráfico:
fig, ax = plt.subplots(figsize=(18, 12))
  
# Gerando o BarPlot Horizontal:
ax.barh(x, y)
  
# Removendo as linhas de limites do gráfico:
for s in ['top', 'bottom', 'left', 'right']:
    ax.spines[s].set_visible(False)
  
# Removendo as marcações dos eixos x e y:
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')
  
# Adicionando preenchimento entre eixos e rótulos:
ax.xaxis.set_tick_params(pad=5)
ax.yaxis.set_tick_params(pad=5)
  
# Adicionando as marcações do grid em x e y:
ax.grid(visible=True, color='grey',
        linestyle='-.', linewidth=0.5,
        alpha=0.2)
  
# Mostrando os valores maiores no alto do gráfico:
ax.invert_yaxis()
  
# Adicionando Rótulos nas Barras do Gráfico
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize=12, fontweight='bold',
             color='grey')
  
# Adicionando um Título ao Gráfico:
ax.set_title('Despesas Totais por Categorias - Período de 2019 a 2021 - valores em Milhões de Reais',
             loc='center', )

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
  
# # Adicionando uma Marca d'água no Gráfico
# fig.text(0.9, 0.15, 'Márcio Estevam da Silva', fontsize=12,
#          color='grey', ha='right', va='bottom',
#          alpha=0.7)
  
# Mostra o gráfico:
plt.show()

## 5 - Quais são os trechos de viagem mais recorrentes entre as despesas de viagem?

In [ ]:
trecho_viagem = dados_mt['txtTrecho'].dropna().value_counts()

trecho_viagem = trecho_viagem.reset_index()

trecho_viagem.rename(columns={'index':'Origem / Destino', 'txtTrecho':'Qtde de Viagens'}, inplace=True)

trecho_viagem = trecho_viagem.head(20)

trecho_viagem

In [ ]:
aeroportos = { 'Siglas':['BSB', 'CGB', 'OPS', 'SMT', 'GRU', 'CGH', 'SXO'],
               'Cidades': ['Brasília', 'Cuiabá', 'Sinop', 'Sorriso', 'Guarulhos', 'São Paulo', 'São Félix do Araguaia'],
               'Estados': ['DF', 'MT', 'MT', 'MT', 'SP', 'SP', 'MT']
             }
aeroportos = pd.DataFrame(aeroportos)

aeroportos

In [ ]:
trecho_viagem['Origem'] = trecho_viagem['Origem / Destino'].str[0:3]
trecho_viagem['Destino'] = trecho_viagem['Origem / Destino'].str[-3:]

In [ ]:
trecho_viagem['Cidade_origem'] = trecho_viagem['Origem'].map(aeroportos.set_index('Siglas')['Cidades'])

In [ ]:
trecho_viagem['Cidade_destino'] = trecho_viagem['Destino'].map(aeroportos.set_index('Siglas')['Cidades'])

In [ ]:
#trecho_viagem.to_csv('trecho_viagem.csv', index=False)

trecho_viagem

## 5.1 - Plotando um gráfico com os trechos mais recorrentes:

In [ ]:
plt.figure(figsize =(17, 6))

sns.set_style("whitegrid")

ax = sns.barplot( x = trecho_viagem['Origem / Destino'], y = trecho_viagem['Qtde de Viagens'],data = trecho_viagem,
                  linewidth=2, palette = "Set2", ci=None)

for i in ax.patches:
    ax.annotate(i.get_height(),
                  (i.get_x() + i.get_width() / 2, i.get_height()),
                  ha='center',
                  va='baseline',
                  fontsize=12,
                  color='black',
                  xytext=(0, 1),
                  textcoords='offset points')

ax.set_xticklabels(ax.get_xticklabels(),rotation = 50)

# Adicionando Título ao gráfico
plt.title("Os 10 Trechos de Viagens mais Recorrentes no Período de 2019 a 2021", loc="center", fontsize=18)
plt.xlabel("Origem / Destino", fontsize=14)
plt.ylabel("Número de vezes", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

## 6 - Qual a média de gastos por partido?

In [ ]:
gasto_partidos = dados_mt[['sgPartido', 'vlrLiquido']].groupby(['sgPartido']).mean()

gasto_partidos_media = gasto_partidos.groupby(['sgPartido']).mean().reset_index().round(2)

gasto_partidos_media = gasto_partidos_media.sort_values(by='vlrLiquido',ascending=False)

#gasto_partidos_media.to_csv('gasto_partidos.csv', index=False)

gasto_partidos_media.style.format({'vlrLiquido':'R$ {:.2F}'}).background_gradient(cmap='Reds')

In [ ]:
g_p = dados_mt[['txNomeParlamentar','sgPartido', 'vlrLiquido']].groupby(['txNomeParlamentar', 'sgPartido']).sum().sort_values('vlrLiquido',ascending=False).round(2).reset_index()

g_p

In [ ]:
plt.figure(figsize =(20, 10))

sns.set_style("whitegrid")

ax = sns.barplot( x = gasto_partidos_media['sgPartido'], y = (gasto_partidos_media['vlrLiquido']),data = gasto_partidos_media,
                  linewidth=2, palette = "Set2", ci=None)

for i in ax.patches:
    ax.annotate(i.get_height(),
                  (i.get_x() + i.get_width() / 2, i.get_height()),
                  ha='center',
                  va='baseline',
                  fontsize=12,
                  color='black',
                  xytext=(0, 1),
                  textcoords='offset points')

ax.set_xticklabels(ax.get_xticklabels(),rotation = 70)

# Adicionando Título ao gráfico
plt.title("Média de Gastos por Partidos - Período de 2019 a 2021", loc="center", fontsize=18)
plt.xlabel("Partidos")
plt.ylabel("Valores em Reais")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

## 7 - Quais são os partidos que mais e que menos consumiram a cota parlamentar?

A Cota para o Exercício da Atividade Parlamentar (CEAP), instituída pelo Ato da Mesa 43/2009, unificou a verba indenizatória (que vigorava desde 2001), a cota de passagens aéreas e a cota postal-telefônica. O valor mensal do benefício deve ser utilizado pelo deputado para custear despesas típicas do exercício do mandato parlamentar, como aluguel de escritório de apoio ao mandato no estado, passagens aéreas, alimentação, aluguel de carro, combustível, entre outras.
<br>Segue tabela demonstrativa da Cota Parlamentar vigente:

<p align="center"><img src="https://static.wixstatic.com/media/fa559d_3b5f6de0221e463db8c19a7d276b64ac~mv2.jpg/v1/fill/w_450,h_300,al_c,q_80,usm_0.66_1.00_0.01/cota%20parlamentar.webp"></p>

<b>OBSERVAÇÃO:</b> O Estado do Mato Grosso possui limite mensal de <b>R$ 39.428,03</b> por deputado federal. 

Primeiramente para ter uma ideia geral das despesas por partidos no período de 2019 a 2021, separamos as colunas 'sgPartido' e 'vlrLiquido' do DataFrame 'dados_mt' e agrupamos a visualização através da coluna 'sgPartido'e exibimos os 10 primeiros resultados ordenados pelo valor em 'vlrLiquido'.

### Partidos que mais utilizaram a cota parlamentar:

In [ ]:
gasto_partidos = dados_mt[['sgPartido', 'sgUF','vlrLiquido']].groupby(['sgPartido']).sum().sort_values('vlrLiquido', ascending = False)

gasto_partidos = gasto_partidos.reset_index()

gasto_partidos_tabela = gasto_partidos.head(5)

gasto_partidos_tabela.style.format({'vlrLiquido':'R$ {:.2F}'}).background_gradient(cmap='Reds')

### Partidos que menos utilizaram a cota parlamentar:

In [ ]:
gasto_partidos = dados_mt[['sgPartido', 'sgUF','vlrLiquido']].groupby(['sgPartido']).sum().sort_values('vlrLiquido', ascending = True)

gasto_partidos = gasto_partidos.reset_index()

gasto_partidos_tabela = gasto_partidos.head(5)

gasto_partidos_tabela.style.format({'vlrLiquido':'R$ {:.2F}'}).background_gradient(cmap='Reds')

Atribuímos o valor da cota referente ao Estado do Mato Grosso:

In [ ]:
cota_mt = 39428.03

In [ ]:
aux_parlamentares = dados_mt[['txNomeParlamentar', 'sgPartido']].drop_duplicates()

parlamentares_partido_mt = aux_parlamentares['sgPartido'].value_counts().to_frame()

parlamentares_partido_mt

total_cota_partido = parlamentares_partido_mt.rename(columns={'sgPartido':'Qtde_parlamentares'})

total_cota_partido['cota_mensal'] = total_cota_partido['Qtde_parlamentares'] * cota_mt

total_cota_partido.style.format({'cota_mensal':'R$ {:.2F}'}).background_gradient(cmap='Reds')

In [ ]:
total_cota_partido["cota_disponibilizada_periodo"] = (cota_mt * 31) * total_cota_partido['Qtde_parlamentares']

total_cota_partido = total_cota_partido.reset_index().rename(columns={'index':'sgPartido'})

total_cota_partido

In [ ]:
gasto_total_partidos = gasto_partidos.groupby('sgPartido').sum().reset_index()

gasto_total_partidos

In [ ]:
uso_da_cota = pd.merge(total_cota_partido, gasto_total_partidos, on=['sgPartido'])

uso_da_cota['Percentual Utilizado (%)'] = (uso_da_cota['vlrLiquido'] / uso_da_cota['cota_disponibilizada_periodo'])*100

uso_da_cota = uso_da_cota.sort_values(by='Percentual Utilizado (%)', ascending=False)

uso_da_cota = pd.DataFrame(uso_da_cota)

#uso_da_cota.to_csv('uso_da_cota.csv', index=False)

uso_da_cota

In [ ]:
x = uso_da_cota['sgPartido']
y = uso_da_cota['Percentual Utilizado (%)']

plt.figure(figsize=(15,6))

plt.bar(x, y, color='r')

plt.xticks(rotation = 30)
plt.xlabel("Partidos")
plt.ylabel("Percentual de Utilização (%)")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(["% Utilizado da Cota"])
plt.title("Percentual de Utilização da Cota parlamentar")

for index,data in enumerate(y.round(2)):
    plt.text(x=index , y =data+1 , s=f"{data}%" , fontdict=dict(fontsize=16))
    
plt.show()

## 8 - Há alguma correlação entre a quantidade de parlamentares por partido e o valor total dos gastos?

Primeiramente fazemos a verificação do gasto total dos partidos no período:

In [ ]:
gasto_partidos = dados_mt[['sgPartido', 'vlrLiquido']].groupby(['sgPartido']).sum().sort_values('vlrLiquido', ascending = False)

gasto_partidos = gasto_partidos.reset_index()

gasto_partidos = gasto_partidos.rename(columns={'vlrLiquido':'Total_gastos'})

gasto_partidos.style.format({'Total_gastos':'R$ {:.2F}'})

Em seguida checamos a quantidade de parlamentares por partido nesse período:

In [ ]:
aux_partido = dados_mt[['sgPartido', 'txNomeParlamentar']].drop_duplicates()

parlamentares_partido = aux_partido.groupby('sgPartido').count().sort_values(by='txNomeParlamentar', ascending=False).reset_index()

parlamentares_partido = parlamentares_partido.rename(columns={'txNomeParlamentar':'Qtde_parlamentares'})

parlamentares_partido

Fizemos então a junção (Pandas merge) entre a tabela de "gasto_partidos" e a tabela "parlamentares_partido", onde a coluna de união será a "sgPartido":

In [ ]:
parlamentares_despesas = pd.merge(gasto_partidos, parlamentares_partido, on=['sgPartido'])

parlamentares_despesas = parlamentares_despesas.sort_values(by='Qtde_parlamentares', ascending=False)

parlamentares_despesas.to_csv('gasto_qtde_parlamentares.csv', index=False)

parlamentares_despesas

Como se tratam de grandezas em escalas diferentes (Total_gastos x Qtde_parlamentares), os dados precisam ser Normalizados (ajustados em uma escala comum), para isso utilizamos o método <b>MinMaxScaler</b> presente na biblioteca Sklearn. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_01 = MinMaxScaler() 

partido_despesa_normalizado = scaler_01.fit_transform(parlamentares_despesas[['Total_gastos', 'Qtde_parlamentares']])

Após a normalização convertemos os dados gerados em dataframe, renomeando as colunas com os nomes originais:

In [ ]:
df_partido_despesa_normalizado = pd.DataFrame(partido_despesa_normalizado)

df_partido_despesa_normalizado.rename(columns={0:'Total_gastos', 1:'Qtde_parlamentares_partido'}, inplace=True)

df_partido_despesa_normalizado.to_csv('gasto_parlamentar_normalizado.csv', index=False)

df_partido_despesa_normalizado['sgPartido'] = parlamentares_despesas['sgPartido']

#df_partido_despesa_normalizado.to_csv('gasto_partido_qtde_normalizado.csv', index=False)

df_partido_despesa_normalizado

Com os dados agora normalizados, aplicamos a função de correlação:

In [ ]:
correlacao = df_partido_despesa_normalizado.corr()

correlacao

><b>OBSERVAÇÃO:</b> Nota-se uma forte correlação positiva entre o Total de gastos com a Quantidade de Parlamentares.

Em seguida plotamos o gráfico de Mapa de Calor, recomendado para exibição da correlação, onde as cores mais intensas confirmam a alta correlação entre as variáveis comparadas.
<br><b>NOTA:</b> Foi ajustada a escala da correlação conforme preconiza a Estatística Probabilística, com a escala variando de -1 a 1.

In [ ]:
a = sns.heatmap(data = correlacao,
            vmin= -1, vmax=1,
            cmap='coolwarm', center=0, annot=True,
            linewidths=1, linecolor="yellow")

a.set_xticklabels(a.get_xmajorticklabels(), fontsize = 14, rotation = 0)
a.set_yticklabels(a.get_ymajorticklabels(), fontsize = 14)
print()

E finalizamos a avaliação com mais uma representação gráfica, desta vez a dispersão dos valores normalizados, com destaque para a linha de tendência no centro do gráfico.

In [ ]:
ax = sns.pairplot(parlamentares_despesas, y_vars='Total_gastos', x_vars=['Qtde_parlamentares'], kind='reg')

ax.figure.set_size_inches(12,6)

ax.fig.suptitle('Dispersão entre as Variáveis', fontsize=20, y=1.1)

print()

<h2>Equipe de Projeto</h2>

<h3>Para acessar o perfil no GitHub de membro da equipe clique na foto e você será redirecionado.</h3>
<table align="center">
    <tr align="center">        
    <td><a href="https://github.com/leticiafelippe"><img src="https://static.wixstatic.com/media/fa559d_7cd4e0d00dc6431e937b93efc00949c9~mv2.jpg/v1/fill/w_189,h_192,al_c,q_80,usm_0.66_1.00_0.01/LETICIA.webp"</a></td>
    <td><a href="https://github.com/Mestevam1976" target="_blank"><img src="https://static.wixstatic.com/media/fa559d_fa2cf4ccc63e4aa6b4139bd9fd83a54c~mv2.jpg/v1/crop/x_74,y_0,w_550,h_595/fill/w_189,h_192,al_c,q_80,usm_0.66_1.00_0.01/marcio.webp"></a></td>
    <td><a href="https://github.com/PHSFernandes"><img src="https://static.wixstatic.com/media/fa559d_32b5fb5b548d47abbbee5f77f35121c9~mv2.jpg/v1/fill/w_189,h_192,al_c,q_80,usm_0.66_1.00_0.01/PH.webp"</a></td>
    <td><a href="https://github.com/Reinaldos"><img src="https://static.wixstatic.com/media/fa559d_6a3934268c4d46d3901ed0e0324b45f0~mv2.jpg/v1/fill/w_189,h_192,al_c,q_80,usm_0.66_1.00_0.01/REINALDO.webp"</a></td>        
    </tr>    
    <tr align="center">
        <td><p align="center"><b>LETÍCIA FELIPPE</b></p></td>
        <td><p align="center"><b>MÁRCIO ESTEVAM DA SILVA</b></p></td>
        <td><p align="center"><b>PAULO H. S. FERNANDES</b></p></td>
        <td><p align="center"><b>REINALDO LEANDRO DE SOUSA</b></p></td>
                
</table>

In [ ]:
pip freeze > requirements.txt